# LangChain - Model Input Output Exercise

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

In [1]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain.llms import OpenAI
from langchain.output_parsers import DatetimeOutputParser
from langchain.chat_models import ChatOpenAI

In [2]:
api_key = open('../../api_key.txt').read()

In [3]:
model = ChatOpenAI(openai_api_key=api_key)

In [4]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        system_template = "You need to write a single quiz question about {topic}. You only return the quiz question"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
        human_template = "{question}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        
        request = chat_prompt.format_prompt(
            topic=topic,
            question="Give me a quiz question where the correct answer is a specific date."
        ).to_messages()
        
        result = model(request)
        return result.content
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        output_parser = DatetimeOutputParser()
        
        system_template = "You will only answer the question with the date"
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
        
        human_template = "Answer the user's question {question}\n{format_instructions}"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
        
        request = chat_prompt.format_prompt(
            question=question,
            format_instructions=output_parser.get_format_instructions()
        ).to_messages()
        
        result = model(request)
        correct_datetime = output_parser.parse(result.content)
        
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)
        
        year = int(input("Enter the year: "))
        month = int(input("Enter the month (1-12): "))
        day = int(input("Enter the day (1-31): "))

        # Create a datetime object
        user_datetime = datetime(year, month, day)

        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        
        difference = user_answer - ai_answer
        formatted_difference = str(difference)
        
        print(f"The difference between the dates is: {formatted_difference}")
        

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [5]:
quiz_bot = HistoryQuiz()

In [6]:
question = quiz_bot.create_history_question(topic='UEFA Champions League 2015')

In [7]:
question

'On which date did the final match of the UEFA Champions League 2015 take place?'

In [8]:
ai_answer = quiz_bot.get_AI_answer(question)

In [9]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(2015, 6, 6, 19, 45)

In [10]:
user_answer = quiz_bot.get_user_answer(question)

On which date did the final match of the UEFA Champions League 2015 take place?
Enter the year: 2015
Enter the month (1-12): 6
Enter the day (1-31): 6


In [12]:
user_answer

datetime.datetime(2015, 6, 6, 0, 0)

In [13]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: -1 day, 4:15:00
